eventuell notwendie Bibliotheken sind zu installieren :<br>
- gmsh sollte systemweit installiert sein
- pip3 install pygmsh --user
- pip3 install vtk --user

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import vtk

import pygmsh
import numpy as np

In [2]:
geom = pygmsh.built_in.Geometry()

# the characteristic size of the triangles
# a value of 0.1 gives about 20 triangles along the vertical axis (length 2)
lcar = 0.05

# we create the initial geometry as a streched ellipse to create
# different scaling lengths (cell sizes) along the different axes
stretch = 5.0

p1 = geom.add_point([stretch, 0.0, 0.0], lcar)
p2 = geom.add_point([0.0, 1.0, 0.0], lcar)
p3 = geom.add_point([-stretch, 0.0, 0.0], lcar)
p4 = geom.add_point([0.0, -1.0, 0.0], lcar)
pc = geom.add_point([0.0, 0.0, 0.0], lcar)
pa = geom.add_point([1.0, 0.0, 0.0], lcar)

e1 = geom.add_ellipse_arc(start=p1, center=pc, point_on_major_axis=pa, end=p2)
e2 = geom.add_ellipse_arc(start=p2, center=pc, point_on_major_axis=pa, end=p3)
e3 = geom.add_ellipse_arc(start=p3, center=pc, point_on_major_axis=pa, end=p4)
e4 = geom.add_ellipse_arc(start=p4, center=pc, point_on_major_axis=pa, end=p1)

ll = geom.add_line_loop([e1,e2,e3,e4])
geom.add_plane_surface(ll)

mesh = pygmsh.generate_mesh(geom, dim=2)

Info    : Running 'gmsh -2 /tmp/tmp1r_e2lkl.geo -format msh -bin -o /tmp/tmpl1d_omm0.msh' [Gmsh 2.15.0, 1 node, max. 1 thread]
Info    : Started on Mon Feb 17 16:28:31 2020
Info    : Reading '/tmp/tmp1r_e2lkl.geo'...
Info    : Done reading '/tmp/tmp1r_e2lkl.geo'
Info    : Finalized high order topology of periodic connections
Info    : Meshing 1D...
Info    : Meshing curve 1 (Ellipse)
Info    : Meshing curve 2 (Ellipse)
Info    : Meshing curve 3 (Ellipse)
Info    : Meshing curve 4 (Ellipse)
Info    : Done meshing 1D (0.088 s)
Info    : Meshing 2D...
Info    : Meshing surface 6 (Plane, Delaunay)
Info    : Done meshing 2D (0.445292 s)
Info    : 8692 vertices 17384 elements
Info    : Writing '/tmp/tmpl1d_omm0.msh'...
Info    : Done writing '/tmp/tmpl1d_omm0.msh'
Info    : Stopped on Mon Feb 17 16:28:31 2020


In [3]:
# we reverse the streching by scaling the coordinates accordingly
points = vtk.vtkPoints()
for p in mesh.points:
    points.InsertNextPoint(p*[1/stretch,1,1])
    
cells = vtk.vtkCellArray()
triangles = mesh.cells['triangle']
for t in triangles:
    cells.InsertNextCell(3, t)

In [8]:
colors = vtk.vtkNamedColors()

meshData = vtk.vtkPolyData()
meshData.SetPoints(points)
meshData.SetPolys(cells)

meshMapper = vtk.vtkPolyDataMapper()
meshMapper.SetInputData(meshData)

meshActor = vtk.vtkActor()
meshActor.SetMapper(meshMapper)
meshActor.GetProperty().SetPointSize(5)
meshActor.GetProperty().SetColor(colors.GetColor3d("Red"))
meshActor.GetProperty().EdgeVisibilityOn()

axesActor = vtk.vtkAxesActor()

renderer = vtk.vtkRenderer()
renderWindow = vtk.vtkRenderWindow()
renderWindow.SetSize(800,600)
renderWindow.AddRenderer(renderer)
renderWindowInteractor = vtk.vtkRenderWindowInteractor()
renderWindowInteractor.SetRenderWindow(renderWindow)
renderWindowInteractor.Initialize()
style = vtk.vtkInteractorStyleTrackballCamera()
renderWindowInteractor.SetInteractorStyle(style)

# Add the actor to the scene
renderer.AddActor(meshActor)
renderer.AddActor(axesActor)
renderer.SetBackground(colors.GetColor3d("SlateGray"))

# Render and interact
renderWindow.Render()
renderWindowInteractor.Start()

# now the interaction is running until we close the window

# cleanup after closing the window
del renderWindow
del renderWindowInteractor